# Olist Data Preparation
*By Rafael LeRoy*

## Introduction

Our goal for this task is to clean and prepare the data for usage in a dashboard.

Our Dashboard will consist of three views:

**Orders:** 
* Which customers ordered the most
* Which geographical states did most orders come from
* How many orders were there per month

**Sellers:**
* Which sellers had the highest sales revenue
* Which geographical states did most sellers come from
* How many sellers were there per month

**Products:**
* Which product categories sold the most
* Which indivisual products sold the most

We will prepare the data in a way that will most suitably accommodate these views.

## Getting an overview of the Data

Before we begin any of the preparation, it is important that we get an overview of the datasets

In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
from googletrans import Translator

pd.set_option('display.max_columns', 500)
plt.rcParams['axes.grid'] = True

customers = pd.read_csv('brazilian-ecommerce/olist_customers_dataset.csv')
sellers = pd.read_csv('brazilian-ecommerce/olist_sellers_dataset.csv')
products = pd.read_csv('brazilian-ecommerce/olist_products_dataset.csv')
order_items = pd.read_csv('brazilian-ecommerce/olist_order_items_dataset.csv')
orders = pd.read_csv('brazilian-ecommerce/olist_orders_dataset.csv')
order_payments = pd.read_csv('brazilian-ecommerce/olist_order_payments_dataset.csv')
reviews = pd.read_csv('brazilian-ecommerce/olist_order_reviews_dataset.csv')

In [15]:
# Lets summarize the null values and duplicate values in our datasets

def series_describer(df):
    null_dic = df.isna().sum().to_dict()
    null_columns = ', '.join([i for i in null_dic if null_dic[i] != 0])
    null_percentage = len(df[df.isnull().any(axis = 1)]) / len(df) * 100
    return [len(df.columns), len(df), df.duplicated().sum(), len(df[df.isnull().any(axis = 1)]), null_percentage, null_columns]

df = pd.DataFrame([series_describer(i) for i in [customers, sellers, products, order_items, orders, order_payments, reviews]],
                  columns = ['Number Of columns', 'Number of Rows', 'Duplicated Rows', 'Number of Nulls','Null_percentage','Null Columns'],
                  index = ['customers', 'sellers', 'products', 'order_items','orders', 'order_payments', 'reviews'])

df.style.background_gradient()

The table above shows us the number of rows with at least one null value. Other than the reviews dataset which we will deal with later, this data seems to have very few null values meaning the data can be used to create an accurate analysis.

## Preparing the products dataset

In [16]:
# First lets view the Null values in this dataset
products.isna().sum()

From the cell above we can see the `product_name_lenght` and `product_description_lenght` are mispelled so we will correct them.

In [17]:
products.rename(columns = {'product_name_lenght': 'product_name_length',
                           'product_description_lenght': 'product_description_length'}, inplace = True)

We can also see the number of null values for every column in our dataset. The last four columns all contain quantative data so the null values can be imputed with either the median or the mean.

Lets explore the best option for imputation.

In [18]:
products_to_plot = products[['product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']]
products_to_plot.plot(subplots=True, layout=(4,4), kind='box', figsize=(20,20), patch_artist = True)
plt.grid()
plt.subplots_adjust(wspace=0.5)
plt.show();

From the boxplots above, we can see the all four of the columns have many outliers as a rightward skew. As we dont want these outliers represented within our average the best option here would be to impute the median.

In [19]:
cols = ['product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']

products[cols] = products[cols].fillna(products[cols].median())

products.isna().sum()

We now have to move on to the columns with the nominal data. This is where things get a bit tricky. As the data is qualitative, we can't impute with the mean or median and we would have to use the mode. One of of the key objects of the task assigned to us is to analyse the performance of the different products. This means that for this analysis the accuracy of the product_category is incredibly important. For this reason I will just drop these rows instead of imputing the mode as I want the data to be as accurate as possible.

In [20]:
products.dropna(inplace = True)
products.isna().sum()

**Now that the null values have been dealt with, we can move onto feature engineering.**

In [21]:
#Lets first take a look at the unique values for every column.
{k: products[k].unique() for k in products.columns}

**The unique values above show us which columns we could engineer:**
 * The values in the `product_category_name` column are in portuguese. We should translate to english

In [22]:
# Cleaning up the product category names to remove the underscores
products['product_category_name'] = products['product_category_name'].str.replace('_', ' ')

#Translating from Portugease to English
#Lets translate the product_category_column from Portugease to English
translator = Translator()
translated_dic = {i: translator.translate(i, dest = 'en').text for i in products['product_category_name'].unique()}
translated_dic

In [23]:
# Some of the values have incorrectly translated so we will change those manually

translated_dic['informatica acessorios'] = 'Computer Accessories'
translated_dic['malas acessorios'] = 'Accessory Bags'
translated_dic['fashion calcados'] = 'Fashion Shoes'
translated_dic['telefonia'] = 'Telephone'
translated_dic['eletroportateis'] = 'Small Appliances'
translated_dic['Climatizacao'] = 'Air Conditioning'
translated_dic['telefonia fixa'] = 'Telephony Fix'
translated_dic['eletrodomesticos 2'] = 'Home Appliances 2'
translated_dic['portateis casa forno e cafe'] = 'Portable Home Oven and Coffee'
translated_dic['portateis cozinha e preparadores de alimentos'] = 'Portable Cooking and Food Preparers'
translated_dic['flores'] = 'Flowers'
translated_dic['moveis colchao e estofado'] = 'mattress and upholstery furniture'

translated_dic =  {k:v.title() for k,v in translated_dic.items()}

products['product_category_name'] = products['product_category_name'].map(translated_dic)

**Finally, we will check for any irregular values that may need to be removed**

If the data has any extreme outliars or any impossible values then they will have to be removed

In [24]:
quantative_rows = products.iloc[:, 2:]
quantative_rows.plot(subplots=True, layout=(4,8), kind='box', figsize=(20,18), patch_artist=True)
plt.subplots_adjust(wspace=0.5)
print(quantative_rows.min(axis=0))

From the Boxplots above we can see that while there are a lot of outliars, given the context of the columns, none are inexplainable. 

When looking at the minimum value for each column however, we can see that `product_weight_g` has a minimum of 0 which should be impossible. Lets explore the `product_weight_g` column to try and solve the problem.

In [25]:
products[products['product_weight_g'] == 0]

The cell above shows every row in which the `product_weight_g` is 0. It is impossible for a physical object to have no weight therefore we will remove these rows.

## Preparing the sellers dataset

In [26]:
# First lets view the Null values in this dataset
sellers.isna().sum()

As there are no null values, we can move straight onto feature engineering.

In [27]:
#Lets first take a look at the unique values for every column.
{k: sellers[k].unique() for k in sellers.columns}

**The unique values above show us which columns we could engineer:**
 * The values in the `seller_state` column have the abbriviations for the state names. To create map charts in Tableau, we would have to replace these abbriviations with the states full name

In [28]:
# Scraping the State names from wikipedia
brazilian_cities_url = 'https://en.wikipedia.org/wiki/Federative_units_of_Brazil'
res = requests.get(brazilian_cities_url)
soup = BeautifulSoup(res.text, "html.parser")
table = soup.find_all('table')
state_names = pd.read_html(str(table[1]))[0]

dic = pd.Series(state_names['Flag and name'].values,index=state_names['Code']).to_dict()
sellers['seller_state'] = sellers['seller_state'].map(dic)

In [29]:
sellers.head()

## Preparing the customers dataset

In [30]:
# First lets view the Null values in this dataset
customers.isna().sum()

As there are no null values, we can move straight onto feature engineering.

In [31]:
# Lets take a look at the unique values for every column.
{k: customers[k].unique() for k in customers.columns}

**The unique values above show us which columns we could engineer:**
 * much like the `seller_state` column the `customer_state` column also has the abbriviations for the state names. We will have to change these
 * Using the `customer_unique_id column`, we could create a boolean column which would be used to find returning customers.

In [32]:
# Using the previously defined dictionary, we can change the customer_state values
customers['customer_state'] = customers['customer_state'].map(dic)
customers['customer_state'].head()

In [33]:
# Creating a column to find returning customers
customers['repeat_customer'] = customers['customer_unique_id'].duplicated(keep = False)

## Preparing the order_items dataset

In [34]:
# # First lets view the Null values in this dataset
order_items.isna().sum()

As there are no null values, we can move straight onto feature engineering.

In [35]:
{k: order_items[k].unique() for k in order_items.columns}

**The unique values above show us which columns we could engineer:**
 * We will first check if the `shipping_limit_date` is in a format that Tableau supports. yyyy-MM-dd HH:mm:ss is supported by Tableau therefore there is no issue with that column.

**Finally, we will check for any irregular values that may need to be removed**

If the data has any extreme outliars or any impossible values then they will have to be removed

In [36]:
years = order_items['shipping_limit_date'].apply(lambda x: x.split('-')[0])
quantative_rows = order_items.loc[:,['price', 'freight_value']]
quantative_rows['Year'] = years

fig, ax = plt.subplots(1, 2, figsize = (16, 4))
sns.histplot(order_items['price'], ax = ax[0], bins = 50)
sns.histplot(order_items['freight_value'], ax = ax[1], bins = 50)
ax[0].axvline(order_items['price'].median(), color = 'red')
ax[1].axvline(order_items['freight_value'].median(), color = 'red')
fig.show()
pd.DataFrame({'min_value': quantative_rows.min(axis = 0), 'max_value' :quantative_rows.max(axis = 0)})

Above we have plotted histograms for `price` and `freight_value` columns. While the data for both columns do have a rightward skew with many outliars, the data tends to show what we would expect with most orders being less that 500 dollars and most shipping costs being below 50 dollars.

We have also displayed the minimum and maximum values of both columns and there does not seem to be any inexplicable values.

## Preparing the orders dataset

In [37]:
# First lets view the Null values in this dataset
orders.isna().sum()

This dataset seems to have a large number of null values. Lets try and work out why this is.

In [38]:
# Viewing the null values in the order_delivered_customer_date column

orders[orders['order_delivered_customer_date'].isna()]['order_status'].value_counts()

From furthur inspection, we can the null values in this dataset fall almost entirely on orders which have not yet been delivered. As you can't provide information on an event that has not yet happened, some of the values have been left blank. As we now know that these nulls aren't due to any error, we can leave them. Having said this however, there is no explination for the nulls values where the `order_status` is delivered so we will remove those rows.

In [39]:
drop_index = orders[(orders['order_status'] == 'delivered') & (orders.isnull().any(axis = 1))].index
orders = orders.drop(drop_index).reindex()

## Preparing the order_payments dataset

In [40]:
# First lets view the Null values in this dataset
order_payments.isna().sum()

The are no null values so lets move onto checking fr any outliars

In [41]:
quantative_rows = order_payments[['payment_sequential', 'payment_installments', 'payment_value']]
quantative_rows.plot(subplots=True, layout=(4,4), kind='box', figsize=(20,20), patch_artist = True)
plt.grid()
plt.subplots_adjust(wspace=0.5)
plt.show();

pd.DataFrame({'min_value': quantative_rows.min(axis = 0), 'max_value' :quantative_rows.max(axis = 0),
              'median_value' :quantative_rows.median(axis = 0), 'mean_value': quantative_rows.mean(axis = 0)})

While our boxplots show many outliars for every column, given the context there shouldn't be any cause for concern.

What should be looked at however is the `payment_value` and `payment_installments` columns having  minimum values of 0.

In [42]:
# Isolate rows for which either payment_sequential or payment_value is 0
order_payments[(order_payments['payment_value'] == 0) | (order_payments['payment_installments'] == 0)]

From the cell above, we can see that for the rows in which the `payment_value` is 0, the `payment_type` is either a voucher or some unspecified payment_type meaning that the customer not spending any money is viable. For that reason, we will leave these hows.

There is no explination however for the `payment_installments` being 0 so for that reason we will drop those rows.

In [43]:
order_payments = order_payments[order_payments['payment_installments'] != 0]

## Preparing the reviews_dataset

In [44]:
# First lets view the Null values in this dataset
reviews.isna().sum()

The null values in this dataframe are only in the `review_comment_title` column and the `review_comment_message` section. It is plausible that leaving a message to go with your review is optional meaning that a lot of people would have just left this section blank. Because of this, we will leave these null values.

In [45]:
answer_year = reviews['review_answer_timestamp'].apply(lambda x: float(x.split('-')[0]))
creation_year = reviews['review_creation_date'].apply(lambda x: float(x.split('-')[0]))

quantative_rows = reviews.loc[:, ['review_score']]
quantative_rows['answer_year'] = answer_year
quantative_rows['creation_year'] = creation_year

pd.DataFrame({'min' : quantative_rows.min(axis = 0), 'max': quantative_rows.max(axis = 0),
             'mean': quantative_rows.mean(axis = 0), 'Median': quantative_rows.median(axis = 0)})

## Merging the Datasets

In [46]:
olist = customers.merge(orders, on = 'customer_id', how = 'left')
olist = olist.merge(order_payments, on = 'order_id', how = 'left')
olist = olist.merge(order_items, on = 'order_id', how = 'left')
olist = olist.merge(products, on = 'product_id', how = 'left')
olist = olist.merge(sellers, on = 'seller_id', how = 'left')
olist = olist.merge(reviews, on = 'order_id', how = 'left')

## We have now successfully prepared the data